<a href="https://colab.research.google.com/github/awaiskhan005/DEEP-LEARNING-AND-FORECASTING/blob/main/BTC_100_paths_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install properscoring

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from sklearn.preprocessing import MinMaxScaler
import json
import datetime
import time
from google.colab import drive
from properscoring import crps_ensemble

# Mount Google Drive
drive.mount('/content/drive')

# Fetch Bitcoin data
def fetch_btc_data(interval='5m', period='30d'):
    btc = yf.download(tickers='BTC-USD', interval=interval, period=period, auto_adjust=True)
    return btc

# Prepare data for LSTM
def prepare_data(df, lookback=50):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df[['Close']])
    X, y = [], []
    for i in range(lookback, len(scaled_data) - 1):
        X.append(scaled_data[i - lookback:i])
        y.append(scaled_data[i, 0])
    return np.array(X), np.array(y), scaler

# Build LSTM model
def build_lstm_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(100, return_sequences=True),
        Dropout(0.2),
        LSTM(100, return_sequences=True),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Predict next 24 hours
def predict_future(model, data, scaler, steps=288, num_paths=100):
    predictions = []
    current_input = data[-1]
    for path_num in range(num_paths):
        print(f"Generating prediction path {path_num+1}/{num_paths}...")
        temp_input = current_input.copy()
        pred_path = []

        for step in range(steps):
            pred = model.predict(temp_input.reshape(1, *temp_input.shape), verbose=0)[0, 0]
            new_row = np.hstack([pred.reshape(-1, 1), temp_input[-1, 1:].reshape(1, -1)])
            pred_price = scaler.inverse_transform(new_row)[0, 0]
            pred_path.append(pred_price)
            temp_input = np.vstack([temp_input[1:], new_row])

            if step % 50 == 0:
                print(f"Step {step}/{steps} for path {path_num+1}...")

        predictions.append(pred_path)

    print("Prediction complete!")
    return np.array(predictions)

# Save predictions to Google Drive
def save_predictions(predictions, timestamps, filename=None):
    if filename is None:
        filename = f'/content/drive/MyDrive/predictions_{datetime.datetime.utcnow().strftime("%Y%m%d_%H%M%S")}.json'
    with open(filename, 'w') as f:
        json.dump({'timestamps': timestamps, 'predictions': predictions.tolist()}, f)
    print(f"Predictions saved in Google Drive: {filename}")
    return filename

# CRPS Calculation Function
def calculate_crps_for_miner(simulation_runs, real_price_path, time_increment):
    scoring_intervals = {
        "5min": 300,
        "30min": 1800,
        "3hour": 10800,
        "24hour_abs": 86400,
    }

    def get_interval_steps(scoring_interval, time_increment):
        return int(scoring_interval / time_increment)

    sum_all_scores = 0.0
    for interval_name, interval_seconds in scoring_intervals.items():
        interval_steps = get_interval_steps(interval_seconds, time_increment)
        simulated_changes = calculate_price_changes_over_intervals(
            simulation_runs, interval_steps, absolute_price=interval_name.endswith("_abs"))
        real_changes = calculate_price_changes_over_intervals(
            real_price_path.reshape(1, -1), interval_steps, absolute_price=interval_name.endswith("_abs"))[0]

        num_intervals = simulated_changes.shape[1]
        crps_values = np.zeros(num_intervals)
        for t in range(num_intervals):
            forecasts = simulated_changes[:, t]
            observation = real_changes[t]
            crps_values[t] = crps_ensemble(observation, forecasts)

        total_crps_interval = np.sum(crps_values)
        sum_all_scores += total_crps_interval

    return sum_all_scores

# Fetch real BTC prices after 24 hours and calculate CRPS
def fetch_real_prices_and_calculate_crps(predictions_file):
    btc_real = fetch_btc_data(interval='5m', period='1d')  # Fetch last 24 hours
    real_prices = btc_real['Close'].values[-288:]

    with open(predictions_file, 'r') as f:
        data = json.load(f)
        predictions = np.array(data['predictions'])
        timestamps = data['timestamps']

    crps_score = calculate_crps_for_miner(predictions, real_prices, 300)
    print(f"CRPS Score: {crps_score}")
    plot_real_vs_predicted(predictions, real_prices, timestamps)
    return crps_score

# Plot predicted vs actual prices
def plot_real_vs_predicted(predictions, real_prices, timestamps):
    fig = go.Figure()
    for i, path in enumerate(predictions):
        fig.add_trace(go.Scatter(x=timestamps, y=path, mode='lines', name=f'Path {i+1}'))
    fig.add_trace(go.Scatter(x=timestamps, y=real_prices, mode='lines', name='Actual Price', line=dict(color='red')))
    fig.update_layout(title='Bitcoin Price Prediction vs Actual', xaxis_title='Time', yaxis_title='Price')
    fig.show()

# Main workflow
btc_data = fetch_btc_data()
X, y, scaler = prepare_data(btc_data)
model = build_lstm_model(X.shape[1:])
model.fit(X, y, epochs=10, batch_size=32, verbose=1)
future_predictions = predict_future(model, X, scaler, num_paths=100)
timestamps = [(datetime.datetime.utcnow() + datetime.timedelta(minutes=5 * i)).isoformat() for i in range(288)]
predictions_filename = save_predictions(future_predictions, timestamps)
print("Predictions saved and plotted successfully!")

# Manually run the below after 24 hours
# crps_score = fetch_real_prices_and_calculate_crps(predictions_filename)


Mounted at /content/drive


[*********************100%***********************]  1 of 1 completed


Epoch 1/10
264/264 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 0.0314
Epoch 2/10
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0026
Epoch 3/10
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0019
Epoch 4/10
264/264 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0012
Epoch 5/10
264/264 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 8.3505e-04
Epoch 6/10
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 6.4356e-04
Epoch 7/10
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 5.7525e-04
Epoch 8/10
264/264 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 5.7790e-04
Epoch 9/10
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 4.9996e-04
Epoch 10/10
264/264 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 5.0561e-04
Generating prediction path 1/100...
Step 0/288 for path 1...
Step 50/288 for path 1...
Step 100/288 for path 1...
Step 150/288 for path 1...
Step 200/288 for path 1...
Step 250/288 for path 1...
Generating prediction path 2/100...
Step 0/288 for path 2...
Step 50/288 for path 2...
Step 100/288

nstructions for Running the Updated Script
1 Run the Full Script Now
Yes, you need to run the entire updated script to:  Fetch BTC data
Train the LSTM model
 Generate 100 prediction paths
 Save the predictions to Google Drive

 Run the entire script in Colab now.

 What to Do After 24 Hours
After 24 hours, you only need to run this single line in Colab

In [1]:
pip install properscoring

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from properscoring import crps_ensemble

In [7]:
def calculate_crps_for_miner(simulation_runs, real_price_path, time_increment):
    scoring_intervals = {
        "5min": 300,
        "30min": 1800,
        "3hour": 10800,
        "24hour_abs": 86400,
    }

    def get_interval_steps(scoring_interval, time_increment):
        return int(scoring_interval / time_increment)

    sum_all_scores = 0.0
    for interval_name, interval_seconds in scoring_intervals.items():
        interval_steps = get_interval_steps(interval_seconds, time_increment)

        # Ensure we only use the valid range of data
        simulated_changes = calculate_price_changes_over_intervals(
            simulation_runs, interval_steps, absolute_price=interval_name.endswith("_abs")
        )
        real_changes = calculate_price_changes_over_intervals(
            real_price_path.reshape(1, -1), interval_steps, absolute_price=interval_name.endswith("_abs")
        )[0]

        num_intervals = min(simulated_changes.shape[1], len(real_changes))  # Adjust to valid size
        crps_values = np.zeros(num_intervals)

        for t in range(num_intervals):
            forecasts = simulated_changes[:, t]
            observation = real_changes[t]
            crps_values[t] = crps_ensemble(observation, forecasts)

        total_crps_interval = np.sum(crps_values)
        sum_all_scores += total_crps_interval

    return sum_all_scores


In [8]:
predictions_filename = "/content/drive/MyDrive/predictions_20250303_102631.json"  # Update if needed
crps_score = fetch_real_prices_and_calculate_crps(predictions_filename)


[*********************100%***********************]  1 of 1 completed


CRPS Score: 35151.02746172808
